# **Applied Machine Learning Homework 5**
**Due 2 May,2022 (Monday) 11:59PM EST**

### Natural Language Processing
We will train a supervised training model to predict if a tweet has a positive or negative sentiment.

####  **Dataset loading & dev/test splits**

**1.1) Load the twitter dataset from NLTK library**

In [ ]:
import nltk
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples 

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


**1.2) Load the positive & negative tweets**

In [ ]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

**1.3) Create a development & test split (80/20 ratio):**

In [ ]:
#code here
from sklearn.model_selection import train_test_split

tweets = []
tweets = all_positive_tweets + all_negative_tweets

print(len(tweets))

y     = []
y_pos = [1]*len(all_positive_tweets)
y_neg = [0]*len(all_negative_tweets)
y     = y_pos + y_neg
print(len(y))


dev_text,test_text,dev_y,test_y = train_test_split(tweets, y, test_size = 0.2, random_state = 42)

10000
10000


#### **Data preprocessing**
We will do some data preprocessing before we tokenize the data. We will remove `#` symbol, hyperlinks, stop words & punctuations from the data. You can use the `re` package in python to find and replace these strings. 

**1.4) Replace the `#` symbol with '' in every tweet**

In [ ]:
#code here
dev_text_1 = [string.replace("#", "") for string in dev_text]
test_text_1 = [string.replace("#", "") for string in test_text]


**1.5) Replace hyperlinks with '' in every tweet**

In [ ]:
import re

dev_text_2 = []
test_text_2 = []

for string in dev_text_1:
    dev_text_2.append(re.sub(r"\bhttp\S+\b", "", string))

for string in test_text_1:
    test_text_2.append(re.sub(r"\bhttp\S+\b", "", string))

**1.6) Remove all stop words**

In [ ]:
#from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.tokenize import sent_tokenize, word_tokenize
#nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')

#from sklearn.feature_extraction.text import CountVectorizer
# vector_stop_words = CountVectorizer(stop_words='english')
# dev_text_3 = vector_stop_words.fit_transform(dev_text_2)
# test_text_3 = vector_stop_words.transform(test_text_2)

stop_words = set(stopwords.words('english'))

dev_text_3 = []
for tweet in dev_text_2:
    text_tokens = word_tokenize(tweet)
    for word in text_tokens:
        tokens_without_sw = [word for word in text_tokens if not word in stop_words]
    filtered_sentence = (" ").join(tokens_without_sw)
    dev_text_3.append(filtered_sentence)

test_text_3 = []
for tweet in test_text_2:
    text_tokens = word_tokenize(tweet)
    for word in text_tokens:
        tokens_without_sw = [word for word in text_tokens if not word in stop_words]
    filtered_sentence = (" ").join(tokens_without_sw)
    test_text_3.append(filtered_sentence)

print(len(dev_text_3))
print(len(test_text_3))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
8000
2000


**1.7) Remove all punctuations**

In [ ]:
#code here
dev_text_4  = []
test_text_4 = []
for string in dev_text_3:
    dev_text_4.append(re.sub(r'[^\w\s]', '', string))

for string in test_text_3:
    test_text_4.append(re.sub(r'[^\w\s]', '', string))


**1.8) Apply stemming on the development & test datasets using Porter algorithm**

In [ ]:
#code here
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

porter = PorterStemmer()
def stemSentence(sentence):
    token_words = word_tokenize(sentence)
    stem_sentence = [porter.stem(word) for word in token_words]
    return " ".join(stem_sentence)

dev_text_5 = [stemSentence(review) for review in dev_text_4]
test_text_5 = [stemSentence(review) for review in test_text_4]

In [ ]:
print(dev_text_5[:3])

['matt', 'lachdog_au posica good thank anyway', 'bf mean']


#### **Model training**

**1.9) Create bag of words features for each tweet in the development dataset**

In [ ]:
#code here
from sklearn.feature_extraction.text import CountVectorizer

vector = CountVectorizer()
dev_X  = vector.fit_transform(dev_text_5)
test_X = vector.transform(test_text_5)

dev_X


<8000x14570 sparse matrix of type '<class 'numpy.int64'>'
	with 52332 stored elements in Compressed Sparse Row format>

**1.10) Train a supervised learning model of choice on the development dataset**

In [ ]:
#code here
from sklearn.linear_model import LogisticRegressionCV

lr = LogisticRegressionCV(max_iter=1000).fit(dev_X,dev_y)
lr.score(test_X,test_y)

0.749

**1.11) Create TF-IDF features for each tweet in the development dataset**

In [ ]:
#code here
from sklearn.feature_extraction.text import TfidfVectorizer

vector = TfidfVectorizer()
dev_X_2  = vector.fit_transform(dev_text_5)
test_X_2 = vector.transform(test_text_5)

**1.12) Train the same supervised learning algorithm on the development dataset with TF-IDF features**

In [ ]:
#code here
lr = LogisticRegressionCV(max_iter=1000).fit(dev_X_2,dev_y)
lr.score(test_X_2,test_y)

0.763

**1.13) Compare the performance of the two models on the test dataset**

In [ ]:
#code here
# The TF-IDF features improve the performance by 1.4% !